In [30]:
import sys, os; sys.path.append(os.path.abspath(".."))
import pandas as pd
import numpy as np
import statsmodels.api as sm

from src.io_utils import load_parquet
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Load prepared data

X_final = load_parquet("../data/interim/X_final.parquet")
transformed_df = load_parquet("../data/interim/transformed_df.parquet")
y = load_parquet("../data/interim/target.parquet")["SalePrice"]

# your existing linear evaluation/tuning helpers can be reused here
# (paste your cv_eval_models + tune/eval code or import your module if you wrapped it)

In [14]:
def univariate_linreg_statsmodels(
    X: pd.DataFrame,
    y: pd.Series,
    *,
    log_target: bool = False
) -> pd.DataFrame:
    """
    Runs separate OLS for each single feature vs. y.
    If log_target=True, fits on log1p(y) and reports RMSE in dollars (back-transformed).
    Coefficients and intercept are reported in the model's native space
    (log-price when log_target=True; raw dollars otherwise).
    """
    rows = []
    y_array = np.asarray(y)
    y_trainable = np.log1p(y_array) if log_target else y_array  # training target

    for col in X.columns:
        x = X[[col]]
        if np.allclose(x.values.std(), 0):
            # constant feature: skip fit/metrics
            rows.append({
                "feature": col,
                "r2": np.nan,
                "rmse": np.nan,
                "coef": 0.0,
                "intercept": float(np.mean(y_trainable)),
                "p_value": np.nan,
                "coef_ci_low": np.nan,
                "coef_ci_high": np.nan,
                "fitted_on_log_target": bool(log_target)
            })
            continue

        Xc = sm.add_constant(x, has_constant="add")
        model = sm.OLS(y_trainable, Xc).fit()

        # Predictions in training space
        y_pred_train_space = model.predict(Xc)

        # RMSE: report in dollars if we trained in log space
        if log_target:
            y_pred_dollars = np.expm1(y_pred_train_space)
            rmse = float(np.sqrt(((y_array - y_pred_dollars) ** 2).mean()))
        else:
            rmse = float(np.sqrt(((y_array - y_pred_train_space) ** 2).mean()))

        coef = float(model.params[col])
        ci_low, ci_high = model.conf_int().loc[col].tolist()
        p = float(model.pvalues[col])

        rows.append({
            "feature": col,
            "r2": float(model.rsquared),
            "rmse": rmse,
            "coef": coef,
            "intercept": float(model.params["const"]),
            "p_value": p,
            "coef_ci_low": float(ci_low),
            "coef_ci_high": float(ci_high),
            "fitted_on_log_target": bool(log_target)
        })

    out = pd.DataFrame(rows).sort_values("r2", ascending=False).reset_index(drop=True)
    return out

In [15]:
results_df = univariate_linreg_statsmodels(X_final, y, log_target=True)

results_df

feature            r2          rmse      coef  intercept  \
0            OverallQual  6.657922e-01  42826.954278  0.315983  12.013480   
1                TotalSF  6.638459e-01  43895.469032  0.315521  12.013480   
2              GrLivArea  5.329337e-01  53117.216109  0.282703  12.013480   
3             TotalBaths  4.488665e-01  60769.294082  0.259450  12.013480   
4              ExterQual  4.436785e-01  54418.774564  0.460922  10.920666   
..                   ...           ...           ...       ...        ...   
262        LotConfig_FR2  5.934953e-07  76150.575135 -0.001742  12.013533   
263    RoofStyle_Mansard  3.421496e-07  76151.250332  0.003645  12.013466   
264  Exterior1st_Plywood  7.390638e-08  76151.740968  0.000399  12.013450   
265    HouseStyle_2.5Unf  2.132083e-09  76151.254731 -0.000194  12.013482   
266             PoolArea           NaN           NaN  0.000000  12.013480   

      p_value  coef_ci_low  coef_ci_high  fitted_on_log_target  
0    0.000000     0.307337      0.324629                  True  
1    0.000000     0.306850      0.324192                  True  
2    0.000000     0.272482      0.292924                  True  
3    0.000000     0.248347      0.270553                  True  
4    0.000000     0.440989      0.480855                  True  
..        ...          ...           ...                   ...  
262  0.968801    -0.089086      0.085602                  True  
263  0.976309    -0.237045      0.244336                  True  
264  0.988988    -0.056315      0.057113                  True  
265  0.998130    -0.162848      0.162459                  True  
266       NaN          NaN           NaN                  True  

[267 rows x 9 columns]

In [17]:
pd.concat([transformed_df, y], axis = 1).groupby(results_df.feature[3])['SalePrice'].mean().plot(kind="bar")

<Axes: xlabel='TotalBaths'>

In [19]:


# --- Helpers ---
def rmse_compat(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)  # works across sklearn versions
    return float(np.sqrt(mse))

def get_feature_names(X):
    if hasattr(X, "columns"):
        return list(X.columns)
    return [f"x{i}" for i in range(X.shape[1])]

# --- Main ---
def fit_multiple_linreg_all_features(
    X, y, *, test_size=0.2, random_state=42, n_splits=5, log_target=False
):
    """
    X: preprocessed feature matrix (DataFrame or ndarray), no NaNs
    y: target Series/array (SalePrice)
    log_target: if True, fits on log1p(y) and reports back-transformed RMSE too
    """
    feature_names = get_feature_names(X)

    # Target transform (optional)
    if log_target:
        y_trainable = np.log1p(y)
    else:
        y_trainable = np.asarray(y)

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_trainable, test_size=test_size, random_state=random_state
    )

    # Fit
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    # Predict
    y_pred_test = lr.predict(X_test)
    y_pred_train = lr.predict(X_train)

    # Metrics (in target space)
    if log_target:
        # Back-transform for error metrics in dollars
        y_test_dollars = np.expm1(y_test)
        y_pred_test_dollars = np.expm1(y_pred_test)
        test_rmse = rmse_compat(y_test_dollars, y_pred_test_dollars)
        test_r2 = r2_score(y_test_dollars, y_pred_test_dollars)
    else:
        test_rmse = rmse_compat(y_test, y_pred_test)
        test_r2 = float(r2_score(y_test, y_pred_test))

    # Cross-validated RMSE (on full data)
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    neg_mse = cross_val_score(lr, X, y_trainable, scoring="neg_mean_squared_error", cv=cv)
    if log_target:
        # Back-transform each fold's MSE is not available; report in log space as approx
        cv_rmse = float(np.sqrt(-neg_mse).mean())
        cv_rmse_std = float(np.sqrt(-neg_mse).std())
    else:
        cv_rmse = float(np.sqrt(-neg_mse).mean())
        cv_rmse_std = float(np.sqrt(-neg_mse).std())

    # Coefficients
    coef_df = pd.DataFrame({
        "feature": feature_names,
        "coef": lr.coef_,
        "abs_coef": np.abs(lr.coef_)
    }).sort_values("abs_coef", ascending=False).reset_index(drop=True)

    metrics = {
        "test_rmse": test_rmse,
        "test_r2": test_r2,
        "cv_rmse_mean": cv_rmse,
        "cv_rmse_std": cv_rmse_std,
        "intercept": float(lr.intercept_),
        "fitted_on_log_target": bool(log_target),
    }

    # Also return out-of-sample predictions (useful for residual checks)
    outputs = {
        "y_test_pred": y_pred_test if not log_target else y_pred_test_dollars,
        "y_test_true": y_test if not log_target else y_test_dollars,
    }

    return lr, coef_df, metrics, outputs

In [41]:
pd.set_option('display.max_rows', 30)
X_final

GrLivArea  LotFrontage   LotArea  MasVnrArea  1stFlrSF  2ndFlrSF  \
0     -1.588620     0.141937 -0.217772   -0.795661 -0.764551 -0.870843   
1     -0.942580    -1.359700 -1.471320    1.127538 -0.115687 -0.870843   
2     -1.091416    -0.257649 -0.749430   -0.795661 -0.265173 -0.870843   
3     -0.973019     0.636082 -0.153439   -0.795661 -1.329908  0.893107   
4      0.525779     0.220791 -0.091576   -0.795661 -0.940801  1.190664   
...         ...          ...       ...         ...       ...       ...   
2575  -1.250895     0.141937  0.014477   -0.795661 -0.425349 -0.870843   
2576   0.653027     0.141937  0.891061   -0.795661  1.486894 -0.870843   
2577   1.111743     0.712955 -0.680796   -0.795661 -0.265173  1.238744   
2578   0.846939     0.141937  0.008096    1.114526 -0.316598  1.191376   
2579   0.963849     0.636082  0.167784    1.060617 -0.067374  1.187437   

      LowQualFinSF  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  \
0        -0.113739   -0.953233    -1.066314      -0.430612  -0.108445   
1        -0.113739   -0.953233     1.105686      -0.430612  -0.108445   
2        -0.113739    0.989840    -1.066314       1.693304   8.456829   
3        -0.113739   -0.953233    -1.066314       2.466196  -0.108445   
4        -0.113739   -0.953233     0.716879      -0.430612  -0.108445   
...            ...         ...          ...            ...        ...   
2575     -0.113739   -0.953233     1.073867      -0.430612  -0.108445   
2576     -0.113739   -0.953233    -1.066314      -0.430612  -0.108445   
2577     -0.113739   -0.953233    -1.066314      -0.430612  -0.108445   
2578     -0.113739    1.076307     1.064361      -0.430612  -0.108445   
2579     -0.113739    1.053819     0.986119      -0.430612  -0.108445   

      ScreenPorch  PoolArea   MiscVal   TotalSF  GrLivArea_per_LotArea  \
0        3.212463       0.0 -0.194979 -1.146573              -0.702626   
1       -0.309165       0.0 -0.194979 -0.463782               0.661897   
2       -0.309165       0.0 -0.194979 -0.908098              -0.127286   
3        2.937576       0.0 -0.194979 -1.718223              -0.505988   
4       -0.309165       0.0 -0.194979  0.091224               0.195301   
...           ...       ...       ...       ...                    ...   
2575     2.246098       0.0 -0.194979 -0.789626              -0.712707   
2576    -0.309165       0.0 -0.194979 -1.105633              -0.514787   
2577    -0.309165       0.0 -0.194979  0.740703               1.305801   
2578    -0.309165       0.0 -0.194979  0.537846               0.295847   
2579    -0.309165       0.0 -0.194979  0.368504               0.212711   

      BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0         1.089555      -0.25377 -1.009265 -0.757892     -1.033650   
1         1.089555      -0.25377  0.823179 -0.757892     -1.033650   
2        -0.838464      -0.25377 -1.009265 -0.757892     -1.033650   
3        -0.838464      -0.25377 -1.009265 -0.757892     -1.033650   
4         1.089555      -0.25377  0.823179  1.245552      0.181855   
...            ...           ...       ...       ...           ...   
2575     -0.838464      -0.25377 -1.009265 -0.757892     -1.033650   
2576     -0.838464      -0.25377  0.823179 -0.757892      1.397359   
2577     -0.838464      -0.25377  0.823179 -0.757892      1.397359   
2578      1.089555      -0.25377  0.823179  1.245552      0.181855   
2579     -0.838464      -0.25377  0.823179  1.245552      0.181855   

      KitchenAbvGr  TotRmsAbvGrd  Fireplaces  GarageCars  TotalBaths  \
0        -0.199051     -1.555298    0.610254    0.342402   -0.258285   
1        -0.199051     -0.903785   -0.931817   -1.010426    0.996660   
2        -0.199051     -0.903785   -0.931817   -1.010426   -1.513230   
3        -0.199051     -0.252272   -0.931817   -1.010426   -1.513230   
4        -0.199051     -0.252272   -0.931817    0.342402    1.624132   
...            ...           ...         ...         ...         ...   
2575  

In [32]:
lr, coef_df, metrics, out = fit_multiple_linreg_all_features(X_final, y, log_target=True)
print(metrics)

{'test_rmse': 23419.76879527101, 'test_r2': 0.9108398173601446, 'cv_rmse_mean': 0.11059793395074613, 'cv_rmse_std': 0.01295385051579095, 'intercept': 10.902362651165404, 'fitted_on_log_target': True}
